# UMAP preparation

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib

import numpy as np
import pandas as pd
import lifelines
import pandas as pd

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import shutil

import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Box

import matplotlib.pyplot as plt
from lifelines import CRCSplineFitter
import warnings
from lifelines.utils import CensoringType

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from IPython.display import clear_output
import pathlib

from lifelines.utils import concordance_index

In [ ]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_name = "210714_metabolomics"
#data_path = "/data/analysis/ag-reils/steinfej"
data_pre = f"{base_path}/data/2_datasets_pre/{project_name}"
data_post = f"{base_path}/data/3_datasets_post/{project_name}"

project_label = "21_metabolomics_multitask"
project_path = f"{base_path}/results/projects/{project_label}"
figures_path = f"{project_path}/figures"
data_results_path = f"{project_path}/data"
pathlib.Path(figures_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(data_results_path).mkdir(parents=True, exist_ok=True)

In [ ]:
endpoints = [
    #Cardiovascular
    'M_MACE',
    'M_coronary_heart_disease',
    'M_cerebral_stroke',
    'M_peripheral_arterial_disease',
    'M_atrial_fibrillation',
    'M_heart_failure',
    'M_abdominal_aortic_aneurysm',
    'M_venous_thrombosis',
    
    # General IM
    'M_type_2_diabetes',
    'M_liver_disease',
    'M_renal_disease',
    
    # Pulmological
    'M_asthma', 
    'M_chronic_obstructuve_pulmonary_disease',  
    
    # Psychiatric/Neurological
    'M_all_cause_dementia',
    'M_parkinsons_disease',   
    
    # Cancers
    "M_lung_cancer",
    "M_non_melanoma_skin_cancer",
    "M_colon_cancer",
    "M_rectal_cancer",
    "M_prostate_cancer",
    "M_breast_cancer",
    
    # Ophtalmological
    'M_cataracts', 
    'M_glaucoma',
    
    # Traumatology
    'M_fractures',
]

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
umap_wide = pd.read_feather(f"{data_results_path}/umap_prep_211026_2.feather").set_index(["endpoint", "eid"]).loc[:, "acetone":]

In [ ]:
import umap

In [ ]:
def calc_umap(endpoint, umap_data):
    reducer = umap.UMAP(min_dist=0.5, n_neighbors=1000)
    embedding_matrix = reducer.fit_transform(umap_data.values)
    embedding_df = pd.DataFrame(data=embedding_matrix, columns=["x", "y"], index=umap_data.index).assign(endpoint=endpoint)
    return embedding_df    

In [ ]:
umap_dfs_raw = [calc_umap(endpoint, umap_wide.loc[(endpoint, )]) for endpoint in tqdm(["M_type_2_diabetes"])]

In [ ]:
umap_df = pd.concat(umap_dfs_raw, axis=0).reset_index()

In [ ]:
umap_df.to_feather(f"{data_results_path}/umap_embeddings_211028.feather")